In [1]:
import json
import keras
import numpy as np

/usr/lib/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
path='./kvret_dataset_public/'
with open(path+'kvret_train_public.json') as f:
    dat=json.load(f)

In [3]:
with open(path+'kvret_dev_public.json') as f:
    valid_dat=json.load(f)

In [4]:
with open(path+'kvret_test_public.json') as f:
    test_dat=json.load(f)

# Analyzing first

In [5]:
from collections import defaultdict
dic = defaultdict(str)
for d in dat:
    for _d in d['dialogue']:
        dic.update(_d['data'].get('slots',{}))

In [6]:
from collections import Counter
dic = set([])
for d in dat:
    for _d in d['dialogue']:
        slots = _d['data'].get('slots')
        if not slots: continue
        dic.update([slots.get('distance')])

In [7]:
from collections import defaultdict
dic = defaultdict(str)
for d in dat:
    for _d in d['dialogue']:
        dic.update(_d.get('data',{}))

In [8]:
requested_slots = list(dic['requested'].keys())

In [9]:
#Kb

In [10]:
d['scenario'].keys()

dict_keys(['kb', 'task', 'uuid'])

In [11]:
s=set([])
for d in dat:
    try:
        keys=d['scenario']['kb']['column_names']
    except:
        continue
    s.update(keys)
print(s)

{'location', 'thursday', 'traffic_info', 'address', 'room', 'party', 'wednesday', 'saturday', 'sunday', 'today', 'friday', 'event', 'time', 'distance', 'date', 'poi', 'monday', 'tuesday', 'poi_type', 'agenda'}


In [12]:
s=set([])
for d in dat:
    try:
        keys=[t['traffic_info'] for t in d['scenario']['kb']['items'] ]
    except:
        continue
    s.update(keys)
print(s)

{'moderate traffic', 'heavy traffic', 'car collision nearby', 'no traffic', 'road block nearby'}


In [13]:
s=set([])
for d in dat:
    try:
        keys=[d['scenario']['kb']['kb_title']]
    except:
        continue
    s.update(keys)
print(s)

{'location information', 'weekly forecast', 'calendar'}


In [14]:
s=set([])
for d in dat:
    try:
        keys=[d['scenario']['task']['intent']]
    except:
        print(d)
        continue
    s.update(keys)
print(s)

{'schedule', 'navigate', 'weather'}


In [15]:
s=defaultdict(set)
for d in dat:
    try:
        s[d['scenario']['kb']['kb_title']].update([d['scenario']['task']['intent']])
    except:
        print(d)
        continue
print(s)

defaultdict(<class 'set'>, {'location information': {'navigate'}, 'calendar': {'schedule'}, 'weekly forecast': {'weather'}})


In [16]:
s=defaultdict(set)
for d in dat:
    try:
        s[d['scenario']['task']['intent']].update(d['scenario']['kb']['column_names'])
    except Exception as e:
        print(e)
        continue
print(s)

defaultdict(<class 'set'>, {'navigate': {'traffic_info', 'address', 'distance', 'poi', 'poi_type'}, 'schedule': {'room', 'party', 'date', 'event', 'time', 'agenda'}, 'weather': {'location', 'thursday', 'wednesday', 'saturday', 'today', 'friday', 'monday', 'tuesday', 'sunday'}})


In [17]:
s=defaultdict(set)
for d in dat:
    try:
        if d['scenario']['task']['intent'] == 'weather':
            print(d)
            print('\n\n\n\n')
            break
    except Exception as e:
        print(e)
        continue
print(s)

{'dialogue': [{'turn': 'driver', 'data': {'end_dialogue': False, 'utterance': 'check forecast for new york'}}, {'turn': 'assistant', 'data': {'end_dialogue': False, 'requested': {'weather_attribute': True, 'date': False, 'location': True}, 'slots': {'location': ' new york'}, 'utterance': 'The weather this week will be hail and rain on Thursday and Sunday, with the lowest temperature being 20F and the highest 90F, Monday will be the only hot day. '}}, {'turn': 'driver', 'data': {'end_dialogue': False, 'utterance': 'Will it be overcast on friday?'}}, {'turn': 'assistant', 'data': {'end_dialogue': False, 'requested': {'weather_attribute': True, 'date': True, 'location': False}, 'slots': {'weather_attribute': 'overcast', 'date': 'Friday'}, 'utterance': 'On Friday it will be cloudy on New York'}}, {'turn': 'driver', 'data': {'end_dialogue': False, 'utterance': 'Thank you!'}}, {'turn': 'assistant', 'data': {'end_dialogue': True, 'requested': {'weather_attribute': False, 'date': False, 'locat

# KB results with beilef states

In [18]:
defaultdict(lambda: 'str')['hi']

'str'

In [19]:
from difflib import SequenceMatcher

def sim(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def close(a,b):
    return sim(a,b)>=0.50

In [20]:
import re
import dateutil

In [21]:
def num_op(s1,op,s2):
    if isinstance(s1, (int, float)):
        num1 = s1
    else:
        m1=re.search(r'([0-9]+)',s1)
        if m1 is None:
            try:
                num1=dateutil.parser.parse(s1)
            except ValueError:
                return False,None
        num1=int(m1.group())
           
    if isinstance(s2,(int,float)):
        num2 = s2
    else:
        m2=re.search(r'([0-9]+)',s2)
        if m2 is None:
            try:
                num2=dateutil.parser.parse(s2)
            except ValueError:
                return False ,None     
        num2=int(m2.group())
    try:
        if op=='equal to':
            return num1==num2,num1
        if op=='greater than':
            return num1>num2,num1
        if op=='less than':
            return num1<num2,num1
    except TypeError:
        return False,num1

In [22]:
def kb_results(kb_data,kb_intent,columns,pred_kb_intent,belief_state,operation):
    """This function gets the kb_data, column names and intent for which the kb is received. 
    If intent identified by the bot is nto same as kb_intent no results will be returned.
    
    intents types: {'schedule', 'weather', 'navigate'}
    
    operation should have same keys as belief state with following possible values:
    str, =, >, <, minimum, maximum  indexed from 0 to 5
    Use None for all the values not numerical. If not None, operation would be performend
    """
    #defaultdict(<class 'set'>, {'navigate': {'poi', 'distance', 'poi_type', 'traffic_info', 'address'},, 'weather': {'thursday', 'sunday', 'today', 'friday', 'wednesday', 'tuesday', 'saturday', 'location', 'monday'}})
    if pred_kb_intent!=kb_intent:
        return [],[]
    if kb_data is None:
        return [],[]
    results = np.ones(len(kb_data))
    confidence = np.ones(len(kb_data))
    # column names possiblity: {'room', 'party', 'event', 'agenda', 'date', 'time'}  
    # Note that date and time are immutable and non-comparable in current dialog, so they are treated as strings
    col_types = defaultdict(lambda: 'str')
    if any(k not in columns for k in belief_state.keys()):
        return [],[]
    for k in belief_state.keys():
        min_idx = None
        min_val = float('Inf')
        max_idx = None
        max_val = -float('Inf')
        for i,items in enumerate(kb_data):
            if results[i] == 0: continue
            if col_types[k]=='str':
                if not (belief_state.get(k) and items.get(k) and operation.get(k) is not None):
                    results[i]=0
                    confidence[i]=0
                else:
                    results[i]=0
                    if operation[k]==0 and close(belief_state[k],items[k]):
                        # Doing string comparison
                        results[i]=1
                        confidence[i] = confidence[i]*sim(belief_state.get(k),items.get(k))
                    elif operation[k]==1:
                        #Doing equal comparison extracting the first number
                        if num_op(belief_state[k],'equal to',items[k])[0]:
                            results[i]=1
                    elif operation[k]==2:
                        #Doing greater than comparison extracting the first number
                        if num_op(belief_state[k],'less than',items[k])[0]:
                            results[i]=1
                    elif operation[k]==3:
                        #Doing less than comparison extracting the first number
                        if num_op(belief_state[k],'greater than',items[k])[0]:
                            results[i]=1
                    elif operation[k]==4:
                        #Doing mimum comparison extracting the first number
                        res,val = num_op(items[k],'less than',(min_val))
                        if res:
                            results[i]=1
                            if min_idx is not None:
                                results[min_idx] = 0
                                confidence[min_idx] = 0
                            min_val = val
                            min_idx = i
                    elif operation[k]==5:
                        #Doing maximum comparison extracting the first number
                        res,val = num_op(items[k],'greater than',(max_val))
                        if res:
                            results[i]=1
                            if max_idx is not None:
                                results[max_idx] = 0
                                confidence[max_idx] = 0
                            max_val = val
                            max_idx = i
    
           
    return np.array(results),np.array(confidence)

In [23]:
kb_data = dat[22]['scenario']['kb']['items']
kb_intent = dat[22]['scenario']['task']['intent']
columns = dat[22]['scenario']['kb']['column_names']

In [24]:
pred_kb_intent = kb_intent
belief_state = {'traffic_info':'no traffic','distance':'6 miles'}
operation = {'traffic_info':0,'distance':5}

In [25]:
kb_results(kb_data,kb_intent,columns,pred_kb_intent,belief_state,operation)

(array([0., 0., 0., 0., 0., 0., 1., 0.]),
 array([0.        , 1.        , 0.        , 1.        , 0.69230769,
        1.        , 0.69565217, 1.        ]))

In [26]:
kb_data[5]

{'distance': '1 miles',
 'traffic_info': 'no traffic',
 'poi_type': 'chinese restaurant',
 'address': '271 Springer Street',
 'poi': 'Mandarin Roots'}

# Preprocessing

In [27]:
from nltk import word_tokenize
from tqdm import tqdm
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

In [28]:
def tokenize(t):
    return word_tokenize(t)

In [29]:
# collection of kb documents
doc_kb=[]
for d in tqdm(dat):
    try:
        for item in d['scenario']['kb']['items']:
            [doc_kb.append(t)  for t in item.values()]
    except TypeError:
        continue

100%|██████████| 2425/2425 [00:00<00:00, 99260.15it/s]


In [30]:
doc_colnames=[]
for d in tqdm(dat):
    try:
        item = d['scenario']['kb']['column_names']
        [doc_colnames.append(t)  for t in item]
    except TypeError:
        continue
    

100%|██████████| 2425/2425 [00:00<00:00, 292569.75it/s]


In [31]:
all_columns=list(set(doc_colnames))

In [32]:
# Vocabulary for the databases
from keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(filters="")

In [33]:
tokenizer.fit_on_texts(doc_kb)

In [34]:
tokenizer.fit_on_texts(["<SOS>","<EOS>"])

In [35]:
DB_VOCAB_LEN = len(tokenizer.word_index)+1

### Also fitting in all the conversations and other things

In [36]:
def all_dict(d):
    texts=[]
    texts.append(" ".join(list(d.keys())))
    for v in d.values():
        if isinstance(v,str):
            texts.append(v) 
        elif isinstance(v,list):
            texts.append(" ".join(all_texts(v)))
        elif isinstance(v,dict):
            texts.append(" ".join(all_dict(v)))
        else:
            try:
                texts.append(str(v))
            except:
                raise Exception(f'type of v is {type(v)}')
    return texts

def all_texts(data):
    texts = []
    for d in data:
        if isinstance(d,dict):
            texts.append(" ".join(all_dict(d)))
        elif isinstance(d,list):
            texts.append(" ".join(all_texts(d)))
        elif isinstance(d,str):
            texts.append(d)
        else:
            try:
                texts.append(str(d))
            except:
                raise Exception(f'type of d is {type(d)}')
    return texts

In [37]:
tokenizer.fit_on_texts(all_texts(valid_dat))

In [38]:
tokenizer.fit_on_texts(all_texts(dat))

In [39]:
tokenizer.fit_on_texts(all_texts(test_dat))

In [40]:
tokenizer.word_index['ok']

625

# Training the db model first

In [41]:
all_columns_wi={a:i for i,a in enumerate(all_columns)}

In [42]:
assert all_columns
assert tokenizer

In [43]:
MAX_QUERIES = 1
NUM_COL = len(all_columns)
CONV_VOCAB_LEN = len(tokenizer.word_index)+1
THRESHOLD = 0.5
MAX_DB_RESULTS = 5
MAX_ENTITY_LENGTH = 10
OPERATOR_LEN = 6
NUM_INTENTS = 3
EMBEDDING_SIZE=50

In [44]:
result = np.zeros((5,6))

In [45]:
# Converting rules based db to desired output first
def results_to_vector(bs_output,pred_intent,operation,kb_data,kb_intent):
    assert bs_output.shape == (NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN)
    assert operation.shape == (NUM_COL,OPERATOR_LEN)
    pred_intent = np.argmax(pred_intent) if max(pred_intent)>THRESHOLD else None
    kb_intent = np.argmax(kb_intent)
    output=np.zeros((MAX_DB_RESULTS,NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN))
    if intent is None:
        return output
    q=bs_output
    op = operation
    op_conf =  np.max(op,axis=-1)
    op_classes = np.argmax(op,axis=-1) 
    op_classes = [_q if _q_conf>THRESHOLD else None for _q,_q_conf in zip(op_classes,op_conf)]

    q_ents = np.argmax(q,axis=-1)
    q_confs = np.max(q,axis=-1)
    q_mask = np.array(q_confs>THRESHOLD,dtype='float32')
    q_ents = q_mask*q_ents
    q_words = [" ".join([tokenizer.index_word[_q] for _q in __q if _q!=0]) for __q in q_ents]
    # Now that q_words and op_classes are known
    bs={}
    operations = {}
    for j,ent in enumerate(q_words):
        if ent is None or ent=="": continue
        bs[all_columns[j]]=ent
        operations[all_columns[j]] = op_classes[j]
    result,confidence = kb_results(kb_data,kb_intent,columns,pred_intent,bs,operations)
    result=np.array(result)
    confidence=np.array(confidence)
    result = result[np.argsort(confidence)[-1::-1]]
    confidence = confidence[np.argsort(confidence)[-1::-1]]
    final_result=[kb_data[_i] for _i,(c,r) in enumerate(zip(confidence,result)) if c>=THRESHOLD and r==1]
    confidence=[confidence[_i] for _i,(c,r) in enumerate(zip(confidence,result)) if c>=THRESHOLD and r==1]
    kb_result = np.zeros((MAX_DB_RESULTS,NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN))
    for j,r in enumerate(final_result):
        if j==MAX_DB_RESULTS: break
        for k,v in r.items():
            kb_result[j,all_columns_wi[k]] = to_categorical(pad_sequences(tokenizer.texts_to_sequences([v]),maxlen=MAX_ENTITY_LENGTH)\
                                                             ,num_classes=CONV_VOCAB_LEN)*confidence[j]

    output = kb_result
    return output

In [46]:
bs_output=np.zeros((NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN))
intent=np.array([1,0,0])
operation = np.zeros((NUM_COL,OPERATOR_LEN))
bs_output[19,0]=to_categorical(tokenizer.texts_to_sequences(['dish'])[0],num_classes=CONV_VOCAB_LEN)
operation[19,0]=1.0
kb_data = dat[0]['scenario']['kb']['items']

In [47]:
results_to_vector(bs_output,intent,operation,kb_data,kb_intent).shape

(5, 20, 10, 6183)

In [48]:
from keras.layers import Input,Dense,LSTM,Embedding,TimeDistributed, RepeatVector, Concatenate,Reshape
from keras.layers import Lambda
from keras.models import Model

In [49]:
import keras.backend as K
import tensorflow as tf

In [50]:
# model for db
bs_input = Input(shape=(MAX_QUERIES,NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN))
intent_input = Input(shape=(MAX_QUERIES,NUM_INTENTS,))
operation_input = Input(shape=(MAX_QUERIES,NUM_COL,OPERATOR_LEN))

bs_proc = TimeDistributed(TimeDistributed(TimeDistributed(Dense(10,activation='sigmoid'))))(bs_input)
LSTM_bs_emb = TimeDistributed(TimeDistributed(LSTM(50,return_sequences=False,return_state=False)))(bs_proc)
rep_intent_input = TimeDistributed(RepeatVector(NUM_COL))(intent_input)
print(LSTM_bs_emb.shape)
all_steps = Concatenate(axis=-1)([LSTM_bs_emb,operation_input,rep_intent_input])
all_steps = Lambda(lambda x: tf.reshape(x,shape=(-1,MAX_QUERIES,NUM_COL*(50+OPERATOR_LEN+NUM_INTENTS))))(all_steps)
encoder_lstm = Dense(50,activation='relu')(all_steps)
encoder_lstm = TimeDistributed(RepeatVector(MAX_DB_RESULTS))(encoder_lstm)

decoder_lstm1 = TimeDistributed(LSTM(50,return_sequences=True))(encoder_lstm)

decoder_lstm1 = Dense(NUM_COL*50,activation='relu')(decoder_lstm1)
decoder_lstm1 = Lambda(lambda x: tf.reshape(x,shape=(-1,MAX_QUERIES,MAX_DB_RESULTS,NUM_COL,50)))(decoder_lstm1)


decoder_lstm2 = TimeDistributed(Lambda(lambda x: K.tile(K.expand_dims(x,axis=-2),[1,1,1,MAX_ENTITY_LENGTH,1])))(decoder_lstm1)
decoder_lstm3 = TimeDistributed(TimeDistributed(TimeDistributed(LSTM(10,return_sequences=True))))(decoder_lstm2)

out = TimeDistributed(TimeDistributed(TimeDistributed(TimeDistributed(Dense(CONV_VOCAB_LEN,activation='softmax')))))(decoder_lstm3)
db_model = Model(inputs=[bs_input,intent_input,operation_input],outputs=[out])

(?, 1, 20, 50)


In [51]:
db_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1, 20, 10, 61 0                                            
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 1, 20, 10, 10 61840       input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1, 3)         0                                            
__________________________________________________________________________________________________
time_distributed_5 (TimeDistrib (None, 1, 20, 50)    12200       time_distributed_3[0][0]         
__________________________________________________________________________________________________
input_3 (I

In [52]:
db_model.compile(optimizer='adam',loss='categorical_crossentropy')

### Preparing training inputs

In [53]:
def softmax(w, t = 1.0):
    e = np.exp(np.array(w) / t)
    dist = e / np.sum(e)
    return dist

In [54]:
intents = ['schedule', 'weather', 'navigate']
def input_generator(batch_size,data=dat):
    batch_data1=[]
    batch_data2=[]
    batch_data3=[]
    target=[]
    random_dat = [data[i] for i in np.random.permutation(len(data))]
    ij=0
    while True:
        ij+=1
        for d in random_dat:
            kb_intent = d['scenario']['task']['intent']
            kb_col_names = d['scenario']['kb']['column_names']
            kb_data = d['scenario']['kb']['items']
            true_vec_intent = np.zeros(NUM_INTENTS)
            true_vec_intent[intents.index(kb_intent)]=1.0
            pred_intent = softmax(np.random.normal(size=NUM_INTENTS,loc=100,scale=5))
            bs_input = np.zeros((NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN))
            operation = np.zeros((NUM_COL,OPERATOR_LEN))
            num_cols_to_have = np.random.randint(NUM_COL)
            num_ents_to_have = [np.random.randint(NUM_COL) for _ in range(num_cols_to_have)]
            for ii in range(num_cols_to_have):
                for j in range(num_ents_to_have[ii]):
                    ix=(np.random.randint(NUM_COL),np.random.randint(MAX_ENTITY_LENGTH))
                    bs_input[ix] = softmax(np.random.normal(size=CONV_VOCAB_LEN,loc=100,scale=5))
                operation[np.random.randint(NUM_COL)] = softmax(np.random.normal(size=OPERATOR_LEN,loc=100,scale=5))
            batch_data1.append([bs_input])
            batch_data2.append([operation])
            batch_data3.append([pred_intent])
            target.append([results_to_vector(bs_input,pred_intent,operation,kb_data,true_vec_intent)])
            
            if len(batch_data1)==batch_size:
                yield [np.array(batch_data1),np.array(batch_data3),np.array(batch_data2)],np.array(target)
                batch_data1=[]
                batch_data2=[]
                batch_data3=[]
                target=[]

In [55]:
a1=next(input_generator(1,dat))

In [56]:
import psutil
def memory_exhausted():
    if memory_percent_available()<=10:
        print("Memory Exhausted")
        exit()
class memCall(keras.callbacks.Callback):
    def on_batch_end(self, batch, logs={}):
        memory_exhausted()
        
def memory_percent_available():
    return psutil.virtual_memory().available/psutil.virtual_memory().total*100


In [57]:
checkpoint=keras.callbacks.ModelCheckpoint('./db_model.h5',save_best_only=True)

In [ ]:
batch_size=1
db_model.fit_generator(input_generator(batch_size),validation_data=input_generator(batch_size,valid_dat),steps_per_epoch=1250,epochs=10,validation_steps=10000,callbacks=[memCall(),checkpoint])

Epoch 1/10


### Testing db_model

In [58]:
db_model.load_weights('db_model_colab.h5')

In [59]:
batch_size=1
db_model.evaluate_generator(input_generator(batch_size,valid_dat),steps=100)

0.03056308627128601

In [60]:
kb_data = dat[22]['scenario']['kb']['items']
kb_intent = dat[22]['scenario']['task']['intent']
columns = dat[22]['scenario']['kb']['column_names']

In [275]:
pred_kb_intent = kb_intent
belief_state = {'traffic_info':'no traffic','distance':'6 miles'}
operation = {'traffic_info':0,'distance':2}

In [276]:
belief_state

{'traffic_info': 'no traffic', 'distance': '6 miles'}

In [277]:
kb_results(kb_data,kb_intent,columns,pred_kb_intent,belief_state,operation)

(array([0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1.        , 1.        , 1.        , 0.69230769, 0.69565217,
        0.69565217, 1.        , 1.        ]))

In [278]:
tokenizer.texts_to_sequences(['no traffic'])

[[65, 20]]

In [324]:
all_columns_wi

{'location': 0,
 'thursday': 1,
 'traffic_info': 2,
 'address': 3,
 'room': 4,
 'party': 5,
 'wednesday': 6,
 'saturday': 7,
 'sunday': 8,
 'today': 9,
 'friday': 10,
 'event': 11,
 'time': 12,
 'distance': 13,
 'date': 14,
 'poi': 15,
 'monday': 16,
 'tuesday': 17,
 'poi_type': 18,
 'agenda': 19}

In [326]:
bs_output=np.zeros((NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN))
intent=np.array([0,0,1])
operation = np.zeros((NUM_COL,OPERATOR_LEN))
bs_output[13,0:2]=to_categorical(tokenizer.texts_to_sequences(['6 miles'])[0],num_classes=CONV_VOCAB_LEN)
operation[13] = np.array([1,0,0,0,0,0])
kb_data = dat[0]['scenario']['kb']['items']

In [327]:
act=results_to_vector(bs_output,intent,operation,kb_data,intent)

In [328]:
pred=db_model.predict([np.array([[bs_output]]),np.array([[intent]]),np.array([[operation]])])

In [329]:
pred.shape

(1, 1, 5, 20, 10, 6183)

In [332]:
np.argmax(pred[0][0][0],axis=-1),np.max(pred[0][0][0],axis=-1)

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0.23864628, 0.5206451 , 0.5605651 , 0.5658833 , 0.5666025 ,
         0.56670004, 0.5667132 , 0.5667149 , 0.5667153 , 0.56671524],
        [0.23871124, 0.5206845 , 0.56057453, 0.56588525, 0.

In [333]:
np.argmax(act[0],axis=-1)

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0, 130, 131,  76],
       [  0,   0,   0,   0,   0,   0,   0, 334, 153,  68],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  89,  12],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 305, 111],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0

In [321]:
a1=next(input_generator(1,dat))

In [301]:
np.argmax(a1[0][0][0][0][0][-1]),np.max(a1[0][0][0][0][0][-1])

(0, 0.0)

## Memory usage

In [302]:
from pympler import muppy
from pympler import summary
all_objects = muppy.get_objects()
sum1 = summary.summarize(all_objects)


In [64]:
def get_model_memory_usage(batch_size, model):
    import numpy as np
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in set(model.trainable_weights)])
    non_trainable_count = np.sum([K.count_params(p) for p in set(model.non_trainable_weights)])

    total_memory = 4.0*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3)
    return gbytes

In [65]:
get_model_memory_usage(16,db_model)

0.463

In [66]:
from pympler import asizeof

In [67]:
asizeof.asizeof(a1)/1024/1024

56.60871887207031

In [68]:
asizeof.asizeof(db_model.layers[0])/1024/1024

TypeError: 'LazyConfigValue' object is not callable

# Defining the complete model now

This model would receive the belief states and give out the sentences in the coeherent form

In [ ]:
MAX_SEQ_LEN = 30
LATENT_DIM = 50
CONV_VOCAB_LEN = 30000

In [ ]:
from keras.engine.network import Network
def get_model(db_m):
    frozen_db_m = Network(db_m.input,db_m.output,name='frozen_db_model')
    
    text_in = Input(shape=(MAX_SEQ_LEN,EMBEDDING_SIZE))
    text_tok_in = Input(shape=(MAX_SEQ_LEN,CONV_VOCAB_LEN))
    emb_state_in_h = Input(shape=(LATENT_DIM,))
    emb_state_in_c = Input(shape=(LATENT_DIM,))
    
    encoding,emb_state_out_h,emb_state_out_c = LSTM(LATENT_DIM,\
                                                    return_state=True)(text_in,\
                                            initial_state=[emb_state_in_h,emb_state_in_c])
    
    
    
    db_decoder_1 = Lambda(lambda x: K.tile(K.expand_dims(x,axis=-2),[1,MAX_QUERIES,1]))(encoding)
    db_decoder_2 = LSTM(NUM_COL*LATENT_DIM,return_sequences=True)(db_decoder_1)
    db_decoder_2 = Lambda(lambda x: tf.reshape(x,[-1,MAX_QUERIES,NUM_COL,LATENT_DIM]))(db_decoder_2)
    
    db_decoder_3 = Lambda(lambda x: K.tile(K.expand_dims(x,axis=-2),[1,1,1,MAX_ENTITY_LENGTH,1]))(db_decoder_2)
    db = TimeDistributed(TimeDistributed(LSTM(LATENT_DIM,return_sequences=True)))(db_decoder_3)
    bs_out = Dense(MAX_SEQ_LEN,activation='softmax')(db)
    text_tok_in_reshape = Lambda(lambda x: tf.reshape(x,[-1,1,1,1,MAX_SEQ_LEN,CONV_VOCAB_LEN]))(text_tok_in)
    
    bs_out = Lambda(lambda x: K.expand_dims(x))(bs_out)
    print(bs_out.shape,text_tok_in_reshape.shape)
    bs_out = Lambda(lambda x: tf.reduce_sum(x[0]*x[1],axis=-2))([bs_out,text_tok_in])
    print(bs_out.shape)
    op_out = Dense(LATENT_DIM,activation='relu')(db_decoder_2)
    op_out = Dense(OPERATOR_LEN,activation='softmax')(op_out)
    print(op_out.shape)
    intent_out = Dense(LATENT_DIM,activation='relu')(encoding)
    intent_out = Dense(NUM_INTENTS,activation='softmax')(intent_out)
    intent_out = RepeatVector(MAX_QUERIES)(intent_out)
    model_bs = Model(inputs=[text_in,emb_state_in_h,emb_state_in_c,text_tok_in],outputs=[bs_out,intent_out,op_out])
    # DECODER
    print(bs_out,intent_out,op_out)
    decoder_input_db = frozen_db_m([bs_out,intent_out,op_out])
    other_bs_input = encoding
    
    #Mdecoder_input_b is of length None,MAX_QUERIES,MAX_DB_RESULTS,NUM_COL,MAX_ENTITY_LENGTH,CONV_VOCAB_LEN
    dbout_encoder_maxent = TimeDistributed(TimeDistributed(TimeDistributed(LSTM(LATENT_DIM))))(decoder_input_db)
    dbout_encoder_ncol =  Lambda(lambda x: tf.reshape(x,[-1,MAX_QUERIES,MAX_DB_RESULTS,NUM_COL*LATENT_DIM]))(dbout_encoder_maxent)
    dbout_encoder_ncol = Dense(LATENT_DIM,activation='relu')(dbout_encoder_ncol)
    dbout_encoder_dbr = TimeDistributed(LSTM(LATENT_DIM))(dbout_encoder_ncol)
    other_bs_input_rep = RepeatVector(MAX_QUERIES)(other_bs_input)
    dbout_encoder = Concatenate(axis=-1)([dbout_encoder_dbr,other_bs_input_rep])
    _,dbout_s,dbout_h = LSTM(LATENT_DIM,return_state=True)(dbout_encoder)
    decoder_hidden_inputs = [dbout_s,dbout_h]
    
    decoder_inputs = Input(shape=(MAX_SEQ_LEN,EMBEDDING_SIZE))
    print(decoder_inputs.shape,decoder_hidden_inputs[0].shape,decoder_hidden_inputs[1].shape)
    decoder_LSTM = LSTM(LATENT_DIM,return_sequences=True,return_state=True)
    decoder_outputs,_,_ = decoder_LSTM(decoder_inputs,initial_state=decoder_hidden_inputs)
    decoder_outputs = Dense(CONV_VOCAB_LEN,activation='softmax')(decoder_outputs)
    
    full_model = Model(inputs=[text_in,emb_state_in_h,emb_state_in_c,text_tok_in,decoder_inputs],outputs=[decoder_outputs])
    full_model.compile(loss='categorical_crossentropy',optimizer='adam')
    return full_model
    

In [ ]:
full_model=get_model(db_model)

In [ ]:
full_model.summary()

In [ ]:
inp = Input((10,vocab_len))
encoder = LSTM(50,return_state=True)
out,out_h,out_c=encoder(inp)
out = Dense(vocab_len,activation='softmax')(out)
model_rnn_2=Model(inputs=[inp],outputs=out)

out_3 = frozen_db_model(model_rnn_2.output)
out_35 = RepeatVector(10)(out_3)
decoder = LSTM(50,return_sequences=True,return_state=True)
out_4,_,_= decoder(out_35,initial_state=[out_h,out_c])
out_4 = Dense(vocab_len,activation='softmax')(out_4)
model_4 = Model(inputs=[model_rnn_2.input],outputs=[model_rnn_2.output,out_4])